# Agenda, day 2

1. Recap and Q&A
2. Magic methods / "dunder" methods
3. Class attributes
4. Finding attributes with ICPO
5. Inheritance -- what it is, and how ICPO influences it
6. Three models for method inheritance
7. Data inheritance
8. What next?

# Recap

How do we solve problems? It's easiest if we can use a data structure that is appropriate for solving that problem. Objects allow us to create a data structure that is specifically designed to solve a problem. Even though we might be using ints, strings, lists, and dicts to implement our object, the fact that we can think about it at a higher level makes it easier to work with.

The whole idea of object-oriented programming is thus: Create new data structures, along with methods, that together make it easier to think about and use these structures and thus solve our problems.

1. Methods are defined as part of the class body. And actually, methods are attributes on the class. However, we invoke them via the instance, and Python does a little magical substitution, turning the instance into the first argument passed to the method.
2. The first parameter in every method is traditionally called `self`, and gets that instance assigned to it.
3. If you try to invoke a method that doesn't exist, you'll get an "attribute error," with Python telling you that the method doesn't exist.
4. We define new "classes," factories for our data structures. We start the class definition with the reserved word `class`, then the name we want to give it, and then any number of methods that we might want to invoke on our data.
5. If we have a `Person` class, then it creates *instances* of `Person`. Every object has a `type`, and we can invoke `type()` on it to find out its class. In this way, `class` and `type` are two ways to say the same thing.
6. The most important method in a class is `__init__`, the initializer. (We pronounce it "dunder init," because it has a double underscore before and after the word "init.") Its job is to add attributes to a new object after that object is created, but before it's returned to the caller. Typically, we'll assign many attributes based on parameters, but we can add fewer or more, depending on what we want to do. In theory, you can add attributes to an object whenever you want... but it's a good idea to only add new attributes in `__init__`, for easier maintenance.
7. Each time we invoke a method on an instance, the first argument is the instance itself, and is assigned to `self`. You can use another word instead of `self`, and if you forget to make `self` the first parameter, then whatever is the first parameter will get the instance assigned to it. But you really should use `self`.
8. Python doesn't have protected or private status for attributes. (Other languages do, and they call attributes "instance variables" when they're set on `self`.) Whereas other languages have getters and setters so that people don't access those values directly, Python *encourages* us to use them directly!
   

In [1]:
class Person:
    def __init__(self, name, shoe_size):
        self.name = name
        self.shoe_size = shoe_size
        self.bank_balance = 0   # attribute that I set, with a default/starter value, without any connection to a parameter

    def greet(self):
        return f'Hello, {self.name}!'  # if I just say "name", and not "self.name", then Python will not find my attribute

p = Person('Reuven', 46)    

In [2]:
p.greet()   # the same as Person.greet(p)

'Hello, Reuven!'

In [3]:
Person.greet(p)

'Hello, Reuven!'

In [4]:
p.name

'Reuven'

In [5]:
p.name = 'whatever'

In [6]:
p.name

'whatever'

In [7]:
p.greet()

'Hello, whatever!'

In [8]:
p.bank_balance

0

# Exercise: Cellphone

1. Define a `Cellphone` class. Each instance will have two attributes:
    - `number`
    - `model`
2. You should be able to invoke the `call` method on an instance of `Cellphone. It'll return a string saying, "Calling" and then print the number that is calling. You'll have to provide an argument, the number to call.

In [14]:
class Cellphone:
    def __init__(self, number, model):
        self.number = number
        self.model = model

    def call(self, other_number):
        return f'Phone {self.number} is calling {other_number}'

c1 = Cellphone(12345, 'iPhone')
c2 = Cellphone(67890, 'Android')

c1.call(2468)

'Phone 12345 is calling 2468'

In [15]:
c2.call(c1.number)  

'Phone 67890 is calling 12345'

In [10]:
type(c1)

__main__.Cellphone

In [11]:
vars(c1)

{'number': 12345, 'model': 'iPhone'}

In [12]:
type(c2)

__main__.Cellphone

In [13]:
vars(c2)

{'number': 67890, 'model': 'Android'}

In [16]:
# what happens if I print my cellphones?

print(c1)

In [17]:
print(c2)

# Magic methods

When we perform certain operations in Python, behind the scenes, the operator is translated into a method call. For example, when we compare values with `==`, there's really a method that is being invoked.

Every operator that you can think of, plus many you probably cannot, are all implemented using methods.

The thing is, these methods don't always need to be defined. And you probably shoudn't be invoking them on your own very often. Plus, if we define these methods, then we really change the way that our objects work, because Python sees the method and uses it .

For all of these reasons, these special methods, known as "magic methods," have special names. All of their names are "dunders," meaning that they start and end with a double underscore, `__`. (That's not `_`, but rather `__` both before and after the method name.)

I want to show you some magic methods now, and we'll use them in a number of classes.

It's typical to implement a number of these methods. Also, new Python functionality is generally implemented by adding new dunder methods.

You should *not* call a m